## 2021, 2025 csv 파일 concat 전 전처리 코드

1. 각 파일에서 연도 추출하여 GameID에 붙이기
2. 날짜 변환 및 정렬
3. 문자형 컬럼을 숫자형 컬럼으로 변환
4. 승/패/무 + 홈/어웨이 정보 저장
5. 무승부 제거 (이진 분류 예측을 위함)
6. 팀 이름 숫자로 인코딩

In [ ]:
import pandas as pd

def preprocess_baseball_data_and_save(file_paths, output_path):
    # 여러 csv 파일을 읽어 하나의 DataFrame으로 합침
    df_total = pd.concat([pd.read_csv(fp) for fp in file_paths], ignore_index=True)

    # 각 파일에서 연도 추출하여 GameID에 붙이기
    for fp in file_paths:
        year = fp.split('_')[0]
        mask = df_total['GameID'].isin(pd.read_csv(fp)['GameID'])
        df_total.loc[mask, 'GameID'] = year + '-' + df_total.loc[mask, 'GameID']

    # 날짜 변환 및 정렬
    df_total['GameDate'] = pd.to_datetime(df_total['GameID'], errors='coerce')


    # 문자열 분할하여 숫자형 컬럼으로 변환
    df_total[['GO', 'FO']] = df_total['GO-FO'].str.split('-', expand=True).astype(int)
    df_total[['NP_total', 'NP_strike']] = df_total['NP-S'].str.split('-', expand=True).astype(int)
    df_total[['IR', 'IS']] = df_total['IR-IS'].str.split('-', expand=True).astype(int)
    df_total.drop(columns=['GO-FO', 'NP-S', 'IR-IS'], inplace=True)

    # 승/패/무 생성 + 홈/어웨이 정보 저장
    results = []
    home_away = []
    for i in range(0, len(df_total), 2):
        r1 = df_total.loc[i, 'R']
        r2 = df_total.loc[i+1, 'R']
        if r1 > r2:
            results.extend([1, 0])
        elif r1 < r2:
            results.extend([0, 1])
        else:
            results.extend([-1, -1])
        home_away.extend([0, 1])
    df_total['Result'] = results
    df_total['home_away'] = home_away

    # 무승부 인덱스
    draw_indices = df_total[df_total['Result'] == -1].index

    # 짝수면 +1, 홀수면 -1 해서 짝 찾기
    paired_indices = [(i + 1 if i % 2 == 0 else i - 1) for i in draw_indices]

    # union해서 중복 없이 모두 제거
    all_indices_to_remove = set(draw_indices).union(paired_indices)

    # 제거
    df_total = df_total.drop(all_indices_to_remove).reset_index(drop=True)


    # GameID 제거, GameDate를 맨 앞으로
    df_total.drop(columns=['GameID'], inplace=True)
    cols = ['GameDate'] + [col for col in df_total.columns if col != 'GameDate']
    df_total = df_total[cols]

    # 팀 이름 숫자로 인코딩
    team_mapping = {
        'KIA': 0, '두산': 1, '삼성': 2, '키움': 3, '롯데': 4,
        'SSG': 5, '한화': 6, 'KT': 7, 'LG': 8, 'NC': 9
    }
    df_total['Team'] = df_total['Team'].replace(team_mapping)

    df_total = df_total.sort_values('GameDate', kind='stable').reset_index(drop=True)

    # 저장
    df_total.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"저장 완료: {output_path}")

# 사용 예시
file_paths = ['2025_06_01_total.csv']  # 또는 실제 경로로 변경
output_path = 'processed_baseball_2025_06.csv'
preprocess_baseball_data_and_save(file_paths, output_path)


## 2022 csv 파일 concat 전 전처리 코드

1. 각 파일에서 연도 추출하여 GameID에 붙이기
2. 날짜 변환 및 정렬
3. 문자형 컬럼을 숫자형 컬럼으로 변환
4. 승/패/무 + 홈/어웨이 정보 저장
5. 무승부 제거 (이진 분류 예측을 위함)
6. 팀 이름 숫자로 인코딩

In [ ]:
import pandas as pd

def preprocess_baseball_data_and_save(file_paths, output_path):
    import re

    # 날짜 문자열 정제 함수
    def clean_korean_date(date_str):
        return re.sub(r'(\d{4})-(\d{2})월 (\d{2})일', r'\1-\2-\3', date_str)

    # 숫자-숫자 포맷으로 바꾸는 함수 ('9월 1일' → '9-1')
    def convert_korean_to_dash_format(value):
        match = re.match(r'(\d{1,2})월 (\d{1,2})일', str(value))
        if match:
            return f"{int(match.group(1))}-{int(match.group(2))}"
        return value

    # 여러 csv 파일을 읽어 하나의 DataFrame으로 합침
    df_total = pd.concat([pd.read_csv(fp) for fp in file_paths], ignore_index=True)

    # 각 파일에서 연도 추출하여 GameID에 붙이기
    for fp in file_paths:
        year = fp.split('_')[0]
        mask = df_total['GameID'].isin(pd.read_csv(fp)['GameID'])
        df_total.loc[mask, 'GameID'] = year + '-' + df_total.loc[mask, 'GameID']

    # GameID 날짜 문자열 정제 및 datetime 변환
    df_total['GameID'] = df_total['GameID'].apply(clean_korean_date)
    df_total['GameDate'] = pd.to_datetime(df_total['GameID'], errors='coerce')
    

    # GO-FO, NP-S, IR-IS → 한글일 경우 숫자-숫자 형식으로 변환
    for col in ['GO-FO', 'NP-S', 'IR-IS']:
        df_total[col] = df_total[col].apply(convert_korean_to_dash_format)

    # 분할하여 정수 컬럼으로 변환
    df_total[['GO', 'FO']] = df_total['GO-FO'].str.split('-', expand=True).astype(int)
    df_total[['NP_total', 'NP_strike']] = df_total['NP-S'].str.split('-', expand=True).astype(int)
    df_total[['IR', 'IS']] = df_total['IR-IS'].str.split('-', expand=True).astype(int)
    df_total.drop(columns=['GO-FO', 'NP-S', 'IR-IS'], inplace=True)

    # GameID 제거, GameDate 앞으로
    df_total.drop(columns=['GameID'], inplace=True)
    cols = ['GameDate'] + [col for col in df_total.columns if col != 'GameDate']
    df_total = df_total[cols]

    # 승/패/무 생성 + 홈/어웨이 정보 저장
    results = []
    home_away = []
    for i in range(0, len(df_total), 2):
        r1 = df_total.loc[i, 'R']
        r2 = df_total.loc[i+1, 'R']
        if r1 > r2:
            results.extend([1, 0])
        elif r1 < r2:
            results.extend([0, 1])
        else:
            results.extend([-1, -1])
        home_away.extend([0, 1])
    df_total['Result'] = results
    df_total['home_away'] = home_away

    # 무승부 인덱스
    draw_indices = df_total[df_total['Result'] == -1].index

    # 짝수면 +1, 홀수면 -1 해서 짝 찾기
    paired_indices = [(i + 1 if i % 2 == 0 else i - 1) for i in draw_indices]

    # union해서 중복 없이 모두 제거
    all_indices_to_remove = set(draw_indices).union(paired_indices)

    # 제거
    df_total = df_total.drop(all_indices_to_remove).reset_index(drop=True)

    # 팀 이름 숫자로 인코딩
    team_mapping = {
        'KIA': 0, '두산': 1, '삼성': 2, '키움': 3, '롯데': 4,
        'SSG': 5, '한화': 6, 'KT': 7, 'LG': 8, 'NC': 9
    }
    df_total['Team'] = df_total['Team'].replace(team_mapping)

    df_total = df_total.sort_values('GameDate', kind= 'stable').reset_index(drop=True)

    # 저장
    df_total.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"저장 완료: {output_path}")

# 실행 코드
file_paths = ['2022_total.csv']
output_path = 'processed_baseball_2022.csv'
preprocess_baseball_data_and_save(file_paths, output_path)


## 2023 csv 파일 concat 전 전처리 코드

1. 각 파일에서 연도 추출하여 GameID에 붙이기
2. 날짜 변환 및 정렬
3. 문자형 컬럼을 숫자형 컬럼으로 변환
4. 승/패/무 + 홈/어웨이 정보 저장
5. 무승부 제거 (이진 분류 예측을 위함)
6. 팀 이름 숫자로 인코딩

In [ ]:
import pandas as pd
import re
import os

def extract_year_from_filename(filepath):
    basename = os.path.basename(filepath)  # 경로에서 파일명만 추출
    match = re.match(r'(\d{4})_', basename)  # '2024_' 형식에만 일치
    if match:
        return match.group(1)
    else:
        raise ValueError(f"파일명에서 연도를 찾을 수 없습니다: {filepath}")
    

# GameID를 날짜 문자열로 변환 ('03월 23일' + '2024' → '2024-03-23')
def convert_korean_gameid_to_date(gameid, year):
    match = re.match(r'0?(\d{1,2})월 0?(\d{1,2})일', str(gameid))
    if match:
        return f"{year}-{int(match.group(1)):02}-{int(match.group(2)):02}"
    return None

# '09월 13일' 같은 날짜를 '9-13' 형식으로 변환
def convert_korean_date_to_numeric(value):
    match = re.match(r'0?(\d{1,2})월 0?(\d{1,2})일', str(value))
    if match:
        return f"{int(match.group(1))}-{int(match.group(2))}"
    return value

# 엑셀에서 망가진 값 예외 처리 ('Oct-85' → '10-13')
def fix_common_garbage_gofo(val):
    if val == 'Oct-85':
        return '10-13'
    return val

# 'x-y' 형식을 안전하게 정수 쌍으로 분리
def safe_split_to_int_pair(val):
    try:
        parts = str(val).split('-')
        if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
            return int(parts[0]), int(parts[1])
    except:
        pass
    return None, None

# 전체 전처리 함수
def preprocess_baseball_data_and_save(file_paths, output_path):
    combined = []

    for fp in file_paths:
        year = extract_year_from_filename(fp)  # 연도 추출
        df = pd.read_csv(fp, encoding='utf-16', sep='\t')
        df['GameDate'] = df['GameID'].apply(lambda x: convert_korean_gameid_to_date(x, year))
        combined.append(df)

    df_total = pd.concat(combined, ignore_index=True)
    df_total['GameDate'] = pd.to_datetime(df_total['GameDate'], errors='coerce')
    #df_total = df_total.sort_values('GameDate').reset_index(drop=True)

    # 날짜형식 처리 + 잘못된 값 보정
    for col in ['GO-FO', 'NP-S', 'IR-IS']:
        df_total[col] = df_total[col].apply(convert_korean_date_to_numeric)
        df_total[col] = df_total[col].apply(fix_common_garbage_gofo)

    # 문자열 쌍 → 숫자형 컬럼으로 분리
    df_total[['GO', 'FO']] = df_total['GO-FO'].apply(lambda x: pd.Series(safe_split_to_int_pair(x)))
    df_total[['NP_total', 'NP_strike']] = df_total['NP-S'].apply(lambda x: pd.Series(safe_split_to_int_pair(x)))
    df_total[['IR', 'IS']] = df_total['IR-IS'].apply(lambda x: pd.Series(safe_split_to_int_pair(x)))
    df_total.drop(columns=['GO-FO', 'NP-S', 'IR-IS'], inplace=True)

   # 승/패/무 생성 + 홈/어웨이 정보 저장
    results = []
    home_away = []
    for i in range(0, len(df_total), 2):
        r1 = df_total.loc[i, 'R']
        r2 = df_total.loc[i+1, 'R']
        if r1 > r2:
            results.extend([1, 0])
        elif r1 < r2:
            results.extend([0, 1])
        else:
            results.extend([-1, -1])
        home_away.extend([0, 1])
    df_total['Result'] = results
    df_total['home_away'] = home_away

    # 무승부 인덱스
    draw_indices = df_total[df_total['Result'] == -1].index

    # 짝수면 +1, 홀수면 -1 해서 짝 찾기
    paired_indices = [(i + 1 if i % 2 == 0 else i - 1) for i in draw_indices]

    # union해서 중복 없이 모두 제거
    all_indices_to_remove = set(draw_indices).union(paired_indices)

    # 제거
    df_total = df_total.drop(all_indices_to_remove).reset_index(drop=True)

    # GameID 제거, GameDate를 맨 앞으로
    df_total.drop(columns=['GameID'], inplace=True)
    cols = ['GameDate'] + [col for col in df_total.columns if col != 'GameDate']
    df_total = df_total[cols]

    # 팀 이름 숫자로 인코딩
    team_mapping = {
        'KIA': 0, '두산': 1, '삼성': 2, '키움': 3, '롯데': 4,
        'SSG': 5, '한화': 6, 'KT': 7, 'LG': 8, 'NC': 9
    }
    df_total['Team'] = df_total['Team'].replace(team_mapping)

    df_total = df_total.sort_values('GameDate', kind='stable').reset_index(drop=True)

    # 저장
    df_total.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"저장 완료: {output_path}")

# 사용 예시
file_paths = ['2023_total.csv']  # 또는 실제 경로로 변경
output_path = 'processed_baseball_2023.csv'
preprocess_baseball_data_and_save(file_paths, output_path)

## 2024 csv 파일 concat 전 전처리 코드

1. 각 파일에서 연도 추출하여 GameID에 붙이기
2. 날짜 변환 및 정렬
3. 문자형 컬럼을 숫자형 컬럼으로 변환
4. 승/패/무 + 홈/어웨이 정보 저장
5. 무승부 제거 (이진 분류 예측을 위함)
6. 팀 이름 숫자로 인코딩

In [ ]:
import pandas as pd
import re

# GameID를 날짜 문자열로 변환 ('03월 23일' + '2024' → '2024-03-23')
def convert_korean_gameid_to_date(gameid, year):
    match = re.match(r'0?(\d{1,2})월 0?(\d{1,2})일', str(gameid))
    if match:
        return f"{year}-{int(match.group(1)):02}-{int(match.group(2)):02}"
    return None

# '09월 13일' 같은 날짜를 '9-13' 형식으로 변환
def convert_korean_date_to_numeric(value):
    match = re.match(r'0?(\d{1,2})월 0?(\d{1,2})일', str(value))
    if match:
        return f"{int(match.group(1))}-{int(match.group(2))}"
    return value

# 엑셀에서 망가진 값 예외 처리 ('Oct-85' → '10-13')
def fix_common_garbage_gofo(val):
    if val == 'Oct-85':
        return '10-13'
    return val

# 'x-y' 형식을 안전하게 정수 쌍으로 분리
def safe_split_to_int_pair(val):
    try:
        parts = str(val).split('-')
        if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
            return int(parts[0]), int(parts[1])
    except:
        print("이이이이이이게 머꼬!!!", val)
        pass
    return None, None

# 전체 전처리 함수
def preprocess_baseball_data_and_save(file_paths, output_path):
    combined = []

    for fp in file_paths:
        year = fp.split('_')[0]  # 파일명에서 연도 추출
        df = pd.read_csv(fp, encoding='utf-16', sep='\t')

        # GameID → GameDate 변환
        df['GameDate'] = df['GameID'].apply(lambda x: convert_korean_gameid_to_date(x, year))
        combined.append(df)

    df_total = pd.concat(combined, ignore_index=True)
    df_total['GameDate'] = pd.to_datetime(df_total['GameDate'], errors='coerce')
    

    # 날짜형식 처리 + 잘못된 값 보정
    for col in ['GO-FO', 'NP-S', 'IR-IS']:
        df_total[col] = df_total[col].apply(convert_korean_date_to_numeric)
        df_total[col] = df_total[col].apply(fix_common_garbage_gofo)

    # 문자열 쌍 → 숫자형 컬럼으로 분리
    df_total[['GO', 'FO']] = df_total['GO-FO'].apply(lambda x: pd.Series(safe_split_to_int_pair(x)))
    df_total[['NP_total', 'NP_strike']] = df_total['NP-S'].apply(lambda x: pd.Series(safe_split_to_int_pair(x)))
    df_total[['IR', 'IS']] = df_total['IR-IS'].apply(lambda x: pd.Series(safe_split_to_int_pair(x)))
    df_total.drop(columns=['GO-FO', 'NP-S', 'IR-IS'], inplace=True)

    # 승/패/무 생성 + 홈/어웨이 정보 저장
    results = []
    home_away = []
    for i in range(0, len(df_total), 2):
        r1 = df_total.loc[i, 'R']
        r2 = df_total.loc[i+1, 'R']
        if r1 > r2:
            results.extend([1, 0])
        elif r1 < r2:
            results.extend([0, 1])
        else:
            results.extend([-1, -1])
        home_away.extend([0, 1])
    df_total['Result'] = results
    df_total['home_away'] = home_away

    # 무승부 인덱스
    draw_indices = df_total[df_total['Result'] == -1].index

    # 짝수면 +1, 홀수면 -1 해서 짝 찾기
    paired_indices = [(i + 1 if i % 2 == 0 else i - 1) for i in draw_indices]

    # union해서 중복 없이 모두 제거
    all_indices_to_remove = set(draw_indices).union(paired_indices)

    # 제거
    df_total = df_total.drop(all_indices_to_remove).reset_index(drop=True)


    # 팀 이름 숫자로 인코딩
    team_mapping = {
        'KIA': 0, '두산': 1, '삼성': 2, '키움': 3, '롯데': 4,
        'SSG': 5, '한화': 6, 'KT': 7, 'LG': 8, 'NC': 9
    }
    df_total['Team'] = df_total['Team'].replace(team_mapping)

    # GameID 제거, GameDate를 앞으로 이동
    df_total.drop(columns=['GameID'], inplace=True)
    cols = ['GameDate'] + [col for col in df_total.columns if col != 'GameDate']
    df_total = df_total[cols]

    df_total = df_total.sort_values('GameDate', kind = 'stable').reset_index(drop=True)

    # CSV로 저장 (한글 깨짐 방지용 utf-8-sig)
    df_total.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"저장 완료: {output_path}")

# 사용 예시
file_paths = ['2024_total.csv']
output_path = 'processed_baseball_2024.csv'
preprocess_baseball_data_and_save(file_paths, output_path)


## csv 파일 concat 코드

연도별로 모은 csv 파일을 concat

In [ ]:
import pandas as pd
import re
import os

def preprocess_baseball_data_and_save(file_paths, output_path):
    # 여러 csv 파일을 읽어 하나의 DataFrame으로 합침
    df_total = pd.concat([pd.read_csv(fp) for fp in file_paths], ignore_index=True)

    df_total.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"저장 완료: {output_path}")

# 사용 예시
file_paths = ['processed_baseball_2021.csv', 'processed_baseball_2022.csv', 'processed_baseball_2023.csv', 'processed_baseball_2024.csv', 'processed_baseball_2025.csv', 'processed_baseball_2025_06.csv']  # 또는 실제 경로로 변경
output_path = 'processed_baseball_total.csv'
preprocess_baseball_data_and_save(file_paths, output_path)

## 무승부한 경기가 남아있는지 확인하는 코드

In [ ]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("processed_baseball_total.csv")

# Result == -1 (무승부)인 행 찾기
draw_rows = df[df['Result'] == -1]

# 결과 출력
if not draw_rows.empty:
    print("무승부 경기가 존재합니다!")
    print(draw_rows)
else:
    print("무승부 경기는 존재하지 않습니다.")


## rolling & 최근 전적 column 생성 코드

rolling(5)를 통해 미래의 야구 승패도 rolling을 사용하여 feature을 만들어 적용할 수 있게 함.
최근 전적을 비교하여 야구 승패를 보다 쉽게 예측 가능

In [ ]:
import pandas as pd

def preprocess_and_save_rolling_only(file_paths, output_path):
    # 파일 합치기
    df_total = pd.concat([pd.read_csv(fp) for fp in file_paths], ignore_index=True)


    # 최근 5경기 승/패 전적 계산
    recent_wins = []
    recent_losses = []
    match_history = {}

    for i in range(0, len(df_total), 2):
        row1 = df_total.iloc[i]
        row2 = df_total.iloc[i + 1]

        t1, t2 = row1['Team'], row2['Team']
        key = tuple(sorted((t1, t2)))

        history = match_history.get(key, [])
        wins_t1 = history.count((t1, 1))
        wins_t2 = history.count((t2, 1))

        recent_wins.extend([wins_t1, wins_t2])
        recent_losses.extend([wins_t2, wins_t1])

        if row1['Result'] == 1:
            history.append((t1, 1))
        elif row2['Result'] == 1:
            history.append((t2, 1))

        match_history[key] = history[-5:]

    df_total['Recent_5_Win'] = recent_wins
    df_total['Recent_5_Loss'] = recent_losses

    # 수치형 컬럼에서 제외할 항목
    numeric_cols = df_total.select_dtypes(include='number').columns.tolist()
    exclude_cols = ['Team', 'Result', 'home_away', 'Recent_5_Win', 'Recent_5_Loss',
                    'WPA_p', 'RE24_p', 'WPA_b', 'RE24_b', 'AVG', 'OPS', 
                    'GDP', 'LOB', 'GSC', 'LI_b', 'LI_p', 'IR', 'IS', 'HBP', 'HBP_p'] # 여기에 제외할 열 이름 적으면 됨
    rolling_cols = [col for col in numeric_cols if col not in exclude_cols]

    # 4. 순서 보존 위해 정렬 후 index 저장
    df_total['__original_index'] = df_total.index
    df_total = df_total.sort_values(['Team', 'GameDate'], kind='stable')

    # 5. 팀별로 shift(1) + rolling 적용
    df_rolled = df_total.groupby('Team')[rolling_cols].transform(
        lambda x: x.shift(1).rolling(window=5, min_periods=1).mean()
    )

    # 6. 정렬 복구
    df_total = df_total.sort_values('__original_index').drop(columns='__original_index')
    df_rolled = df_rolled.loc[df_total.index]  # index 정렬 일치시킴

    # # 팀별 rolling 평균
    # df_rolled = (
    #     df_total.groupby('Team')[rolling_cols]
    #     .rolling(window=5, min_periods=1)
    #     .mean()
    #     .reset_index(level=0, drop=True)
    # )

    # # 팀별 rolling 평균 (미래 누설 방지: shift(1))
    # df_rolled = (
    #     df_total.groupby('Team')[rolling_cols]
    #     .apply(lambda x: x.shift(1).rolling(window=5, min_periods=1).mean())
    #     .reset_index(level=0, drop=True)
    # )


    # df_rolled.columns = [f"{col}" for col in df_rolled.columns]

    # 필요한 컬럼만 포함
    df_base = df_total[['GameDate', 'Team']].copy()
    non_rolling_cols = df_total[[col for col in exclude_cols if col != 'Team']]

    # 2. rolling 결과와 함께 병합
    df_result = pd.concat([
        df_base.reset_index(drop=True),
        non_rolling_cols.reset_index(drop=True),
        df_rolled.reset_index(drop=True)
    ], axis=1)

    
    # 컬럼 순서 맞추기
    desired_order = [
        'GameDate', 'Team', 'PA', 'AB', 'R', 'H', 'HR', 'RBI', 'BB', 'HBP', 'SO',
        'GO', 'FO', 'NP', 'GDP', 'LOB', 'AVG', 'OPS', 'LI_b', 'WPA_b', 'RE24_b',
        'IP', 'TBF', 'H_p', 'R_p', 'ER', 'BB_p', 'HBP_p', 'K', 'HR_p', 'GSC',
        'ERA', 'WHIP', 'LI_p', 'WPA_p', 'RE24_p', 'NP_total', 'NP_strike',
        'IR', 'IS', 'home_away', 'Recent_5_Win', 'Recent_5_Loss', 'Result'
    ]
    df_result = df_result[[col for col in desired_order if col in df_result.columns]]

    
    # CSV로 저장
    df_result.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"저장 완료: {output_path}")

# 사용 예시
file_paths = ['processed_baseball_total.csv']
output_path = '../data/rolling_only.csv'
preprocess_and_save_rolling_only(file_paths, output_path)


## 최근 전적 column 생성 코드

rolling(5)를 통해 미래의 야구 승패도 rolling을 사용하여 feature을 만들어 적용할 수 있게 함.
최근 전적을 비교하여 야구 승패를 보다 쉽게 예측 가능

In [ ]:
import pandas as pd

def preprocess_and_save_basic(file_paths, output_path):
    # 1. CSV 파일들 합치기
    df_total = pd.concat([pd.read_csv(fp) for fp in file_paths], ignore_index=True)

    # 2. 최근 5경기 승/패 전적 계산
    recent_wins = []
    recent_losses = []
    match_history = {}

    for i in range(0, len(df_total), 2):
        row1 = df_total.iloc[i]
        row2 = df_total.iloc[i + 1]

        t1, t2 = row1['Team'], row2['Team']
        key = tuple(sorted((t1, t2)))

        history = match_history.get(key, [])
        wins_t1 = history.count((t1, 1))
        wins_t2 = history.count((t2, 1))

        recent_wins.extend([wins_t1, wins_t2])
        recent_losses.extend([wins_t2, wins_t1])

        if row1['Result'] == 1:
            history.append((t1, 1))
        elif row2['Result'] == 1:
            history.append((t2, 1))

        match_history[key] = history[-5:]

    df_total['Recent_5_Win'] = recent_wins
    df_total['Recent_5_Loss'] = recent_losses

    # 3. 컬럼 순서 맞추기
    desired_order = [
        'GameDate', 'Team', 'PA', 'AB', 'R', 'H', 'HR', 'RBI', 'BB', 'HBP', 'SO',
        'GO', 'FO', 'NP', 'GDP', 'LOB', 'AVG', 'OPS', 'LI_b', 'WPA_b', 'RE24_b',
        'IP', 'TBF', 'H_p', 'R_p', 'ER', 'BB_p', 'HBP_p', 'K', 'HR_p', 'GSC',
        'ERA', 'WHIP', 'LI_p', 'WPA_p', 'RE24_p', 'NP_total', 'NP_strike',
        'IR', 'IS', 'home_away', 'Recent_5_Win', 'Recent_5_Loss', 'Result'
    ]
    df_total = df_total[[col for col in desired_order if col in df_total.columns]]

    # 4. CSV로 저장
    df_total.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"저장 완료: {output_path}")

# 사용 예시
file_paths = ['processed_baseball_total.csv']
output_path = '../data/not_rolling_only.csv'
preprocess_and_save_basic(file_paths, output_path)


## rolling_only.csv 특정 column 삭제 코드

correalation matrix와 다중공선성을 계산한 후 필요없는 column들을 삭제

### ⭐ 중요!!! rolling 해서 Nan값이 된 행들도 제거함

In [ ]:
import pandas as pd

# 1. CSV 불러오기
df_total = pd.read_csv('../data/rolling_only.csv')  # ← 여기에 실제 파일명 입력

# 2. 최종 선택된 16개 변수만 선택
selected_columns = ['GameDate', 'Team',
    'OPS', 'BB', 'HBP', 'SO',         # 타자 성과
    'ERA', 'BB_p', 'K', 'IR', 'IS', 'TBF',  # 투수 성과
    'Recent_5_Win', 'Recent_5_Loss',  # 최근 경기 흐름
    'home_away', 'Result'                   # 홈/원정 정보
]

# 3. 해당 변수만 남기고 나머지 제거
df_selected = df_total[selected_columns].copy()

df_cleaned = df_selected.dropna().reset_index(drop=True)

# 4. 결과를 새 파일로 저장
df_cleaned.to_csv('../data/final_data_rolling.csv', index=False, encoding='utf-8-sig')  # ← 저장 파일명 지정

print("16개 변수 선택 및 저장 완료!")

## not_rolling_only.csv 특정 column 삭제 코드

correalation matrix와 다중공선성을 계산한 후 필요없는 column들을 삭제

### ⭐ 중요!!! rolling 해서 Nan값이 된 행들도 제거함

In [ ]:
import pandas as pd

# 1. CSV 불러오기
df_total = pd.read_csv('../data/not_rolling_only.csv')  # ← 여기에 실제 파일명 입력

# 2. 최종 선택된 16개 변수만 선택
selected_columns = ['GameDate', 'Team',
    'OPS', 'BB', 'HBP', 'SO',         # 타자 성과
    'ERA', 'BB_p', 'K', 'IR', 'IS', 'TBF',  # 투수 성과
    'Recent_5_Win', 'Recent_5_Loss',  # 최근 경기 흐름
    'home_away', 'Result'                   # 홈/원정 정보
]

# 3. 해당 변수만 남기고 나머지 제거
df_selected = df_total[selected_columns].copy()

df_cleaned = df_selected.dropna().reset_index(drop=True)

# 4. 결과를 새 파일로 저장
df_cleaned.to_csv('../data/final_data_not_rolling.csv', index=False, encoding='utf-8-sig')  # ← 저장 파일명 지정

print("16개 변수 선택 및 저장 완료!")

## 팀 당 row를 경기 당 row로로

합쳐용~

In [ ]:
import pandas as pd

# 데이터 불러오기
df_new = pd.read_csv("../data/final_data_rolling.csv")

games = []
for i in range(0, len(df_new), 2):

    if df_new.loc[i, 'home_away'] == 0:
        home = df_new.iloc[i+1]
        away = df_new.iloc[i]
    else:
        home = df_new.iloc[i]
        away = df_new.iloc[i + 1]

    # 새로운 경기 단위 row 생성
    game_row = {
        'GameDate': home['GameDate'],
        'home_Team': home['Team'],
        'away_Team': away['Team'],
        'Result': int(home['Result'] > away['Result']) , # 홈팀이 이기면 1, 지면 0
        'home_Recent_5_Win': home['Recent_5_Win'],
        'home_Recent_5_Loss': home['Recent_5_Loss']
    }

    # 제외할 열
    excluded_cols = ['GameDate', 'Team', 'Result','home_away','Recent_5_Win','Recent_5_Loss']

    # 홈팀 feature 추가
    for col in df_new.columns:
        if col not in excluded_cols:
            game_row[f'home_{col}'] = home[col]

    # 원정팀 feature 추가
    for col in df_new.columns:
        if col not in excluded_cols:
            game_row[f'away_{col}'] = away[col]

    games.append(game_row)

# 새로운 데이터프레임으로 변환
df_game_unit = pd.DataFrame(games)

df_game_unit.to_csv('../data/real_final.csv', index=False, encoding='utf-8-sig')  # ← 저장 파일명 지정

print("컬럼 붙이기 완료!")